<a href="https://colab.research.google.com/github/uoacapstonegroup6/CapstoneUOATeam6/blob/main/Next_steps_Soft_Voting_with_Entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import entropy

def compute_entropy(image_slice):
    """Calculate the entropy of a 2D image slice based on pixel intensity."""
    # Flatten the image slice to a 1D array
    pixel_values = image_slice.flatten()

    # Compute the histogram of pixel values
    hist, _ = np.histogram(pixel_values, bins=256, range=(0, 255), density=True)

    # Compute entropy using scipy
    slice_entropy = entropy(hist, base=2)  # Use base-2 entropy
    return slice_entropy

def normalize_entropies(entropies):
    """Normalize the entropy values so that they sum up to 1."""
    total_entropy = np.sum(entropies)
    if total_entropy == 0:
        return np.ones_like(entropies) / len(entropies)  # If all entropies are 0, return equal weights
    return entropies / total_entropy

def weighted_voting(predictions, entropies):
    """
    Perform soft voting with weighted probabilities based on entropy.

    Args:
    predictions: List of arrays, where each array is the class probabilities for a slice.
                 Shape of each array is (num_classes,)
    entropies: List of entropy values for each corresponding slice.

    Returns:
    final_prediction: The predicted class for the 3D image.
    """
    num_slices = len(predictions)
    num_classes = len(predictions[0])

    # Normalize entropies
    normalized_entropies = normalize_entropies(entropies)

    # Initialize array for weighted probabilities
    weighted_probs = np.zeros(num_classes)

    # Sum weighted probabilities from each slice
    for i in range(num_slices):
        weighted_probs += normalized_entropies[i] * predictions[i]

    # Final prediction is the class with the highest weighted probability
    final_prediction = np.argmax(weighted_probs)

    return final_prediction

# Example Usage
if __name__ == "__main__":
    num_slices = 64
    num_classes = 2  # 3 for fracture mnist

    # Simulate some image slices (replace with actual 64x64 image data)
    image_slices = [np.random.randint(0, 256, (64, 64)) for _ in range(num_slices)]

    # Simulate class probability predictions for each slice (replace with actual model outputs)
    predictions = [np.random.rand(num_classes) for _ in range(num_slices)]
    predictions = [p / np.sum(p) for p in predictions]  # Ensure probabilities sum to 1

    # Compute entropy for each slice
    entropies = [compute_entropy(slice) for slice in image_slices]

    # Perform weighted voting
    final_class = weighted_voting(predictions, entropies)

    print(f"Final predicted class for the 3D image: {final_class}")
